# PyPSA Statistics Module Training

A guided ~40-min session exploring `n.statistics` — PyPSA's high-level API for
querying costs, capacities, energy flows, and market metrics from optimized networks.

We use `pypsa.examples.carbon_management()`, a sector-coupled European energy system
from a [Nature Energy paper](https://www.nature.com/articles/s41560-025-01752-6) on
H₂/CO₂ network strategies (2164 buses, 89 carriers, 20 days at 3h resolution).

# Motivation

Imagine a collegue sends you a pypsa network, that you should inspect.... things that you might want to do in the start is **understanding** the network.

In [ ]:
import pypsa
import pandas as pd

n = pypsa.examples.carbon_management()

In [ ]:
n

In [ ]:
n.buses.carrier.value_counts()

**Potential Chain of Thought:**

---

*How should I inspect this? there is so much going on...*


*Okay, let's start with `n.generators.p_nom.grouby(n.generators.carrier)` and then do that for links*

*and then stores - ahh(!) stores have a different attribute `n.stores.e_nom`*

*Ah wait it is `p_nom_opt` / `e_nom_opt` anyway that I should look into......*

---

Then you realize, you've been there multiple times and you remember that this can be cumbersome, often error-prone and verbose. 

This is why we created the **statistics** module.


# Statistics Module


The **summary table** `n.statistics()` gives a quick overview of all metrics at once.

In [ ]:
n.statistics()

`n.statistics` provides a consistent, high-level API that handles component iteration, port mapping, and carrier grouping automatically.
(`n.stats` is available as a shorthand alias for `n.statistics`.)

Each method can be called individually or explored via the summary table:

| Category | Methods |
|---|---|
| **Costs** | `capex()`, `installed_capex()`, `expanded_capex()`, `opex()`, `system_cost()` |
| **Capacity** | `installed_capacity()`, `optimal_capacity()`, `expanded_capacity()`, `capacity_factor()` |
| **Energy** | `supply()`, `withdrawal()`, `energy_balance()`, `transmission()`, `curtailment()` |
| **Market** | `prices()`, `revenue()`, `market_value()` |

Every method accepts the same filtering and grouping parameters:

| Parameter | Description |
|---|---|
| `groupby` | String, list, or callable — how to group results (default: `"carrier"`) |
| `groupby_method` | Aggregation function (`"sum"` (default), `"mean"`, …) |
| `groupby_time` | `"sum"`, `"mean"`, or `False` for time series — default varies by method |
| `components` | Filter to specific component types |
| `carrier` | Filter by carrier name (internal name) |
| `bus_carrier` | Filter by the carrier of the bus |
| `nice_names` | Use human-readable carrier names (default: `True`) |

Note: `prices()` has a simplified interface — `groupby` and `groupby_time` are booleans,
and it does not accept `carrier` or `components` (harmonized in upcoming v1.1).

That's a lot of info, let's see it in action! 

## Cost Analysis

Three cost methods form a hierarchy: 

- **capex** (capital = installed_capex + expanded_capex)
- **opex** (operational)
- **system_cost** (capex + opex combined)

Additionally, `installed_capex` and `expanded_capex` break down capital costs
by existing vs newly built capacity.

In [ ]:
capex = n.statistics.capex() # returns a series
capex

In [ ]:
opex = n.statistics.opex()
opex.head(10)

In [ ]:
system_cost = n.statistics.system_cost()
print(f"Total system cost: {system_cost.sum() / 1e9:.1f} bn €")

The `groupby_method` parameter controls how values are aggregated within each group.
Compare `"sum"` (default, total) vs `"mean"` (average per asset). Let's also use pandas `.style` accessor for better cost formatting.

In [ ]:
costs = pd.concat([
    n.statistics.capex(groupby_method="sum").rename("sum"),
    n.statistics.capex(groupby_method="mean").rename("mean"),
], axis=1)
    
costs.sort_values(by="sum", ascending=False).div(1e6).style.format("{:,.0f} M€")

Beyond `groupby_method`, the `groupby` parameter itself controls how results are grouped.
PyPSA provides built-in groupers — `bus`, `bus_carrier`, `carrier`, `country`, `location`, `name`, `unit` — that you can pass as a single string or a list into `groupby`:

In [ ]:
print("Available groupers:", list(pypsa.statistics.groupers.list_groupers().keys()))

Let's say you are interested in the the country allocation of technologies: 

In [ ]:
n.statistics.capex(groupby=["country", "carrier"]).head(10)

## Capacity Analysis

There are three capacity methods: 

1. `installed_capacity` - existing before optimization
2. `optimal_capacity` - post-optimization
3. `expanded_capacity` - newly built = optimal − installed

in most cases after running an optimization, you want to run the `optimal_capacity` function. 


Let's have a look:

In [ ]:
caps = n.statistics.optimal_capacity()
caps.to_frame().head(10)

Remember there is different capacity:

| Capacity Type | Components |
|---|---|
| Dispatch only | `Generator`, `Link`, `Line`, `Transformer` |
| Dispatch or storage | `Storage Unit` |
| Storage only | `Store` |

Per default, the capacity function return in terms of dispatch capacity. In order to get storage capacities use:

In [ ]:
storage_cap = n.stats.optimal_capacity(storage=True)
storage_cap.to_frame("Capacity").div(1e6).style.format("{:,.0f} TWh")

## Capacity Factors

Capacity factors relative to the optimal capacity are calculated by the `capacity_factor` function which returns the average operation per unit. 

In [ ]:
cf = n.statistics.capacity_factor()
cf.sort_values(ascending=False).dropna()

Now, let's say you want to filter by technologies which produce into the high voltage "AC" electricity-sector. You can do so by setting a `bus_carrier` argument: 

In [ ]:
n.statistics.capacity_factor(bus_carrier="AC").to_frame("CF @ AC")

## Energy Flows

Now that we know what's built and what it costs, the next question is: how does energy flow through the system?

- `supply()` / `withdrawal()` — one-directional production / consumption  
- `energy_balance()` — net flow (positive = supply, negative = withdrawal)
- `transmission()` — energy through branch components (links, lines)
- `curtailment()` — available generation capacity that went unused

In [ ]:
n.statistics.supply().sort_values(ascending=False).head(10)

In [ ]:
n.statistics.withdrawal().sort_values().head(10)

`energy_balance` combines both views — positive values mean net supply, negative mean net consumption. Its default groupby is `["carrier", "bus_carrier"]`:

In [ ]:
eb = n.statistics.energy_balance()
eb.head(15)

We can use pandas `.xs()` to slice a specific bus carrier. Let's look at the AC electricity sector:

In [ ]:
ac_balance = eb.xs("AC", level="bus_carrier")
ac_balance.droplevel(0).sort_values().plot.barh(figsize=(8, 5), xlabel="MWh")

In [ ]:
n.statistics.transmission()

In [ ]:
curt = n.statistics.curtailment()
curt[curt > 0].sort_values(ascending=False)

We saw `energy_balance` defaults to `groupby=["carrier", "bus_carrier"]`. Let's try other groupings:

In [ ]:
n.statistics.energy_balance(groupby="bus_carrier")

Use `components` to restrict which component types are included:

In [ ]:
n.statistics.supply(components=["Generator", "Link"]).head(10)

Set `groupby_time=False` to get a full time series (MW per snapshot) instead of a single aggregated value.
You can also use `groupby_time="mean"` for average MW or `groupby_time="sum"` (default) for total MWh:

In [ ]:
ts = n.statistics.energy_balance(groupby_time=False)
print(f"Shape: {ts.shape} — rows are (component, carrier), columns are timestamps")
ts.iloc[:5, :5]

In [ ]:
pd.concat([
    n.statistics.energy_balance(groupby_time="mean").rename("mean (MW)"),
    n.statistics.energy_balance(groupby_time="sum").rename("sum (MWh)"),
], axis=1).head(10).style.format("{:,.0f}")

## Market Metrics

Now that we understand physical flows, we can look at the economic side:

- `prices()` — marginal prices per bus (shadow prices of the energy balance constraint)
- `revenue()` — income earned by each carrier  
- `market_value()` — revenue per unit of energy (€/MWh), i.e. the average price a technology receives

In [ ]:
prices = n.statistics.prices(groupby="bus_carrier")
print(f"Mean electricity price: {prices.mean():.2f} €/MWh")
prices.plot.barh(figsize=(6, 6), edgecolor="white", xlabel="€/MWh", ylabel="Bus Carrier")

In [ ]:
n.statistics.revenue().sort_values(ascending=False).head(10)

`market_value` = revenue / supply — the average price a carrier receives per MWh. Technologies with high output during low-price hours tend to have lower market values (value deflation):

In [ ]:
n.statistics.market_value().sort_values(ascending=False).head(10)

## Custom Groupers

Register a custom grouper via `groupers.add_grouper(name, func)`. The function
signature is `(n, c, port) -> pd.Series` where `c` is the component name.

In [ ]:
from pypsa.statistics import groupers
def tech_type(n, c, port=""):
    carriers = n.c[c].static["carrier"]
    nice_names = carriers.map(n.carriers.nice_name)
    conditions = {
        "Renewable": ["wind", "solar", "ror", "hydro"],
        "Conventional": ["gas", "oil", "nuclear"],
        "Storage": ["battery", "storage", "reservoir"],
    }
    def classify(name):
        lower = name.lower()
        for label, keywords in conditions.items():
            if any(kw in lower for kw in keywords):
                return label
        return "Other"
    return nice_names.map(classify).rename("tech_type")

groupers.add_grouper("tech_type", tech_type)

In [ ]:
n.statistics.capex(groupby="tech_type")

In [ ]:
n.statistics.supply(groupby=["tech_type", "carrier"], aggregate_across_components=True).to_frame("Supply").head(70)

## Tipps and tricks:

- use `s = n.statistics` in the beginning of your code; then `s.capex(**kwargs)` is very compact
- set `aggregate_accross_components=True` if you have distinct carriers
- specify `bus_carrier` if you deal with sector-coupled networks, so you don't mix units, especially useful for multi-port links

## Recap

| Category | Methods |
|---|---|
| **Costs** | `capex()`, `installed_capex()`, `expanded_capex()`, `opex()`, `system_cost()` |
| **Capacity** | `installed_capacity()`, `optimal_capacity()`, `expanded_capacity()`, `capacity_factor()` |
| **Energy** | `supply()`, `withdrawal()`, `energy_balance()`, `transmission()`, `curtailment()` |
| **Market** | `prices()`, `revenue()`, `market_value()` |
| **Overview** | `n.statistics()` (summary table) |
| **Plotting** | `.plot.bar()`, `.iplot.area()`, etc. on every method |

Key parameters: `groupby`, `groupby_method`, `groupby_time`, `components`, `carrier`, `bus_carrier`, `nice_names`.

**Docs**: [pypsa.org/latest/user-guide/statistics](https://docs.pypsa.org/latest/user-guide/statistics/)

---

# Spoiler for next session: Plotting from Statistics

Every statistics method has `.plot` (matplotlib/seaborn) and `.iplot` (plotly) accessors.
All statistics parameters (`carrier`, `bus_carrier`, `groupby`, ...) pass through — no need to query first and plot separately.

Available plot types: `bar`, `line`, `area`, `box`, `scatter`, `histogram`, `violin`, `chart`, `map`.

In [ ]:
n.statistics.optimal_capacity.iplot.bar()

All statistics parameters pass through. For example, filter by `carrier`:

In [ ]:
n.statistics.supply.iplot.bar(carrier=["onwind", "solar", "offwind"])

For interactive plots, use `.iplot` (plotly) instead of `.plot` (matplotlib):

In [ ]:
n.statistics.energy_balance.iplot.area(
    x="snapshot",
    title="Energy Balance Time Series (interactive)",
)